# Install transformers and nlp library

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.3 MB/s eta 0:00:00


# Import tokenizer and model

In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np

# Load dataset

In [3]:
dataset = load_dataset('csv', data_files='./drive/MyDrive/cn-spam-detection/output.csv', split='train')
type(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

datasets.arrow_dataset.Dataset

# Process dataset

In [4]:
dataset = dataset.train_test_split(test_size=0.3)
dataset
train_set = dataset['train']
test_set = dataset['test']

# Download BERT model

In [5]:
model = BertForSequenceClassification.from_pretrained('bert-base-chinese')

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Create tokenizer

In [6]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [7]:
def preprocess(data):
    return tokenizer(data['text'], padding=True, truncation=True)

train_set = train_set.map(preprocess, batched=True, batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

# Train model

## Define parameters

In [8]:
batchSize = 8
epochs = 5
warmupSteps = 500
weightDecay = 0.01
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batchSize,
    per_device_eval_batch_size=batchSize,
    warmup_steps=warmupSteps,
    weight_decay=weightDecay,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,0.104000,0.072050
2,0.069400,0.109814
3,0.035900,0.075755
4,0.018300,0.078018
5,0.013500,0.064773


{'eval_loss': 0.06477342545986176,
 'eval_runtime': 56.4183,
 'eval_samples_per_second': 106.348,
 'eval_steps_per_second': 13.294,
 'epoch': 5.0}

In [13]:
trainer.save_model("./drive/MyDrive/cn-spam-detection/model")

In [12]:
model.save_pretrained("./pretrained_model")